In [217]:
import pandas as pd
import numpy as np
import seaborn as sns 


In [218]:
data_fake = pd.read_csv("Fake.csv")
data_True = pd.read_csv("True.csv")

In [219]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [220]:
data_True.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [221]:
data_True = data_True.drop(["title", "subject", "date"], axis=1)
data_fake = data_fake.drop(["title", "subject", "date"], axis=1)

In [222]:
data_fake.head()

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [223]:
data_fake["truefalse"] = 0
data_True["truefalse"] = 1

In [224]:
data_True.head()

,text,truefalse
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [225]:
data_new = pd.concat([data_True, data_fake], axis=0)
data_new = data_new.sample(frac=1) #fraction of samples to be returned = 1
data_new.to_csv("merged.csv")

In [226]:
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords, wordnet
stopwords.words("english")
pass

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yash0\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [227]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download("wordnet")
nltk.download("punkt")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yash0\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yash0\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [229]:
from sklearn.model_selection import train_test_split
X = data_new["text"]
y = data_new["truefalse"]

In [230]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,  random_state=42)

In [231]:
import re

In [232]:
def get_wordnet_pos(tag):
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def wordNetLemmatizer(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) #uppercase non-word character
    text = re.sub('https?://\S+|www\.\S+', '', text) #links
    text = re.sub('<.*?>+', '', text) 
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text) 
    text = re.sub(r'[^\w\s]', '', text)
    wnl = WordNetLemmatizer()
    tokens = word_tokenize(text)
    wordsTags  = nltk.pos_tag(tokens)
    return [wnl.lemmatize(word, pos = get_wordnet_pos(tag)) for word, tag in wordsTags]

In [233]:
ch = "Hello my nmam, eaindian  ???"
print(wordNetLemmatizer(ch))

['hello', 'my', 'nmam', 'eaindian']


In [234]:
vect = TfidfVectorizer( tokenizer=wordNetLemmatizer,stop_words="english")
X_train = vect.fit_transform(X_train)
X_test = vect.transform(X_test)

c:\Users\yash0\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\feature_extraction\text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['far', 'make', 'u'] not in stop_words.
  warnings.warn(


In [235]:
from xgboost import XGBClassifier
xg = XGBClassifier()
xg.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [236]:
y_predxg = xg.predict(X_test)

In [237]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predxg))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4657
           1       1.00      1.00      1.00      4323

    accuracy                           1.00      8980
   macro avg       1.00      1.00      1.00      8980
weighted avg       1.00      1.00      1.00      8980



In [238]:
from sklearn.model_selection import cross_val_score
acc = cross_val_score(xg, X_train, y_train, cv = 10)
print(acc.mean())

0.9974943235346471


In [239]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=10)
rf.fit(X_train, y_train)
y_predrf = rf.predict(X_test)

In [240]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predrf))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      4657
           1       0.98      0.95      0.96      4323

    accuracy                           0.97      8980
   macro avg       0.97      0.97      0.97      8980
weighted avg       0.97      0.97      0.97      8980



In [241]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_predlr = lr.predict(X_test)

In [242]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predlr))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      4657
           1       0.98      0.99      0.98      4323

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

